In [1]:
import pandas as pd
import numpy as np

In [36]:
archivo= 'Calculo y Análisis de emisiones 20230727.xlsx'

In [3]:
regional = pd.read_excel('bne_balance_regional.xlsx')

In [11]:
# Diccionario de coincidencias de nombres invertido para transformar regional_total
coincidencias = {
    'Coque Metalúrgico': 'Coque Mineral',
    'Gas de Alto Horno': 'Gas Altos Hornos',
    'Gas Licuado de Petróleo': 'Gas Licuado',
    'Derivados Industriales de Petróleo': 'D.I. de Petróleo',
    'Energía Hidroeléctrica': 'Energia Hídrica',
    'Gas de Coque': 'Gas Coque'
}
sectores = {
    'Comercial, público y residencial': 'Sector Cmrcl., Púb. Y Residencial',
     'Energía': 'Energético: Consumo Propio',
     'Industria y Minería': 'Industrial y Minero',
     'Transformación': 'Centros de transfornación',
     }

# Función para transformar los nombres en regional_total según el diccionario de coincidencias
def transformar_nombres(row):
    combustible = row['Energético']
    if combustible in coincidencias:
        return coincidencias[combustible]
    return combustible


def transformar_sector(row):
    sector = row['Sector']
    if sector in sectores:
        return sectores[sector]
    return sector

def procesar_BNE_regional(archivo):
    regional = pd.read_excel(archivo,sheet_name='BNE_regional')
    regional_orden = regional.query('anio == 2021')[['nombre_region','cod_region','actividad','subactividad','energetico','tcal']]
    regional_orden.columns = ['nombre_region','cod_region','Sector','Subsector','Energético','Tcal BNE']

    regional_orden['Energético'] = regional_orden.apply(transformar_nombres, axis=1)
    regional_orden['Sector'] = regional_orden.apply(transformar_sector, axis=1)
    
    return regional_orden

# Definir la lista de palabras a mantener en minúsculas
palabras_mantener_min = ['y', 'de', 'por']  # Agrega aquí las palabras que desees mantener en minúsculas

# Función personalizada para capitalizar las palabras, excepto las que están en la lista
def capitalizar_excepto_palabras_min(palabra):
    palabras = palabra.split()
    palabras_capitalizadas = [palabra.capitalize() if palabra.lower() not in palabras_mantener_min else palabra.lower() for palabra in palabras]
    return ' '.join(palabras_capitalizadas)

def calcular_emisiones_regionales(archivo,bne_reg = 'BNE_Regional_2021.csv'):

    #Carga de datos
    BNE_reg = pd.read_csv(bne_reg)
    FE_CO2_df = pd.read_excel(archivo,sheet_name='FE CO2 Data').drop(columns='Poder calorifico inferior TJ/kg')
    FE_CO2_df.columns = ['Energético','Factor de emision CO2 kg/Tj']
    CE_S = pd.read_excel(archivo,sheet_name='CE S.')
    PCI_df = pd.read_excel(archivo,sheet_name='PCI')


    BNE_reg['Sector'] = BNE_reg['Sector'].str.lower()
    BNE_reg['Subsector'] = BNE_reg['Subsector'].str.lower()
    BNE_reg['Energético'] = BNE_reg['Energético'].str.lower()
    FE_CO2_df['Energético'] = FE_CO2_df['Energético'].str.lower()
    CE_S['Energético'] = CE_S['Energético'].str.lower()
    CE_S['Sector'] = CE_S['Sector'].str.lower()
    PCI_df['Energético'] = PCI_df['Energético'].str.lower()


    #Juntamos todo en una sola tabla
    data_merge = BNE_reg.merge(FE_CO2_df, how='left',on='Energético')
    data_merge = data_merge.merge(CE_S,on=['Sector','Energético'],how='left')
    data_merge = data_merge.merge(PCI_df, how='left',on='Energético')
    
    # Capitalizar la primera letra de cada palabra después del merge
    data_merge['Sector'] = data_merge['Sector'].apply(capitalizar_excepto_palabras_min)
    data_merge['Subsector'] = data_merge['Subsector'].apply(capitalizar_excepto_palabras_min)
    data_merge['Energético'] = data_merge['Energético'].apply(capitalizar_excepto_palabras_min)

    #Seleccionamos variables
    BNE_Tcal = data_merge['Tcal BNE']
    FE_CO2 = data_merge['Factor de emision CO2 kg/Tj']
    FE_CH4 = data_merge['FE CH4 (kg/TJ)']
    FE_N2O = data_merge['FE N2O (kg/TJ)']
    PCI = data_merge['ajuste a PCI']


    #Cálculos
    #Energía BNE en TJ
    data_merge['BNE TJ'] = BNE_Tcal * 4.1868

    #Consumo de energía del BNE en TJ
    Consumo_BNE_TJ = BNE_Tcal * 4.184

    #Cálculo de emisiones BNE en (kg)
    Emisiones_BNE_CO2_kg = Consumo_BNE_TJ * FE_CO2
    Emisiones_BNE_CH4_kg = Consumo_BNE_TJ * FE_CH4
    Emisiones_BNE_N2O_kg = Consumo_BNE_TJ * FE_N2O

    #Cálculo de emisiones BNE en (Millt)
    Emisiones_BNE_CO2_Millt = Emisiones_BNE_CO2_kg / 1000000000
    Emisiones_BNE_CH4_Millt = Emisiones_BNE_CH4_kg / 1000000000
    Emisiones_BNE_N2O_Millt = Emisiones_BNE_N2O_kg / 1000000000

    #Cálculo de emisiones BNE en (Millt) con ajuste PCI
    Emisiones_BNE_CO2_Millt_PCI = PCI * Emisiones_BNE_CO2_Millt
    Emisiones_BNE_CH4_Millt_PCI = PCI * Emisiones_BNE_CH4_Millt
    Emisiones_BNE_N2O_Millt_PCI = PCI * Emisiones_BNE_N2O_Millt

    #Cálculo de emisiones CO2eq de BNE en (Millt)
    data_merge['Emisiones BNE CO2eq (Millt)'] = Emisiones_BNE_CO2_Millt_PCI*1 + Emisiones_BNE_CH4_Millt_PCI*25 + Emisiones_BNE_N2O_Millt_PCI*298

    #Creamos una tabla con las columnas de interés
    data_final = data_merge[['nombre_region','cod_region','Sector','Subsector','Energético','Tcal BNE','Emisiones BNE CO2eq (Millt)']]

    data_final = data_final.drop_duplicates()
    data_final = data_final.rename(columns={'Tcal BNE': 'BNE Tcal'})
    data_final.to_csv('Emisiones_regionales_BNE_2021.csv',index=False,encoding='utf-8-sig')
    return

In [50]:
# Definir la lista de palabras a mantener en minúsculas
palabras_mantener_min = ['y', 'de', 'por']  # Agrega aquí las palabras que desees mantener en minúsculas

# Función personalizada para capitalizar las palabras, excepto las que están en la lista
def capitalizar_excepto_palabras_min(palabra):
    palabras = palabra.split()
    palabras_capitalizadas = [palabra.capitalize() if palabra.lower() not in palabras_mantener_min else palabra.lower() for palabra in palabras]
    return ' '.join(palabras_capitalizadas)

def calcular_emisiones_regionales(archivo,bne_reg = 'BNE_Regional_2021.csv'):

    #Carga de datos
    BNE_reg = pd.read_csv(bne_reg)
    FE_CO2_df = pd.read_excel(archivo,sheet_name='FE CO2 Data').drop(columns='Poder calorifico inferior TJ/kg')
    FE_CO2_df.columns = ['Energético','Factor de emision CO2 kg/Tj']
    CE_S = pd.read_excel(archivo,sheet_name='CE S.')
    PCI_df = pd.read_excel(archivo,sheet_name='PCI')


    BNE_reg['Sector'] = BNE_reg['Sector'].str.lower()
    BNE_reg['Subsector'] = BNE_reg['Subsector'].str.lower()
    BNE_reg['Energético'] = BNE_reg['Energético'].str.lower()
    FE_CO2_df['Energético'] = FE_CO2_df['Energético'].str.lower()
    CE_S['Energético'] = CE_S['Energético'].str.lower()
    CE_S['Sector'] = CE_S['Sector'].str.lower()
    PCI_df['Energético'] = PCI_df['Energético'].str.lower()


    #Juntamos todo en una sola tabla
    data_merge = BNE_reg.merge(FE_CO2_df, how='left',on='Energético')
    data_merge = data_merge.merge(CE_S,on=['Sector','Energético'],how='left')
    data_merge = data_merge.merge(PCI_df, how='left',on='Energético')
    
    # Capitalizar la primera letra de cada palabra después del merge
    data_merge['Sector'] = data_merge['Sector'].apply(capitalizar_excepto_palabras_min)
    data_merge['Subsector'] = data_merge['Subsector'].apply(capitalizar_excepto_palabras_min)
    data_merge['Energético'] = data_merge['Energético'].apply(capitalizar_excepto_palabras_min)

    #Seleccionamos variables
    BNE_Tcal = data_merge['Tcal BNE']
    FE_CO2 = data_merge['Factor de emision CO2 kg/Tj']
    FE_CH4 = data_merge['FE CH4 (kg/TJ)']
    FE_N2O = data_merge['FE N2O (kg/TJ)']
    PCI = data_merge['ajuste a PCI']


    #Cálculos
    #Energía BNE en TJ
    data_merge['BNE TJ'] = BNE_Tcal * 4.1868

    #Consumo de energía del BNE en TJ
    Consumo_BNE_TJ = BNE_Tcal * 4.184

    #Cálculo de emisiones BNE en (kg)
    Emisiones_BNE_CO2_kg = Consumo_BNE_TJ * FE_CO2
    Emisiones_BNE_CH4_kg = Consumo_BNE_TJ * FE_CH4
    Emisiones_BNE_N2O_kg = Consumo_BNE_TJ * FE_N2O

    #Cálculo de emisiones BNE en (Millt)
    Emisiones_BNE_CO2_Millt = Emisiones_BNE_CO2_kg / 1000000000
    Emisiones_BNE_CH4_Millt = Emisiones_BNE_CH4_kg / 1000000000
    Emisiones_BNE_N2O_Millt = Emisiones_BNE_N2O_kg / 1000000000

    #Cálculo de emisiones BNE en (Millt) con ajuste PCI
    Emisiones_BNE_CO2_Millt_PCI = PCI * Emisiones_BNE_CO2_Millt
    Emisiones_BNE_CH4_Millt_PCI = PCI * Emisiones_BNE_CH4_Millt
    Emisiones_BNE_N2O_Millt_PCI = PCI * Emisiones_BNE_N2O_Millt

    #Cálculo de emisiones CO2eq de BNE en (Millt)
    data_merge['Emisiones BNE CO2eq (Millt)'] = Emisiones_BNE_CO2_Millt_PCI*1 + Emisiones_BNE_CH4_Millt_PCI*25 + Emisiones_BNE_N2O_Millt_PCI*298

    #Creamos una tabla con las columnas de interés
    data_final = data_merge[['nombre_region','cod_region','Sector','Subsector','Energético','Tcal BNE','Emisiones BNE CO2eq (Millt)']]

    data_final = data_final.drop_duplicates()
    data_final = data_final.rename(columns={'Tcal BNE': 'BNE Tcal'})
    data_final.to_csv('Emisiones_regionales_BNE_2021.csv',index=False,encoding='utf-8-sig')
    return

In [52]:
df = calcular_emisiones_regionales(archivo)
df

,nombre_region,cod_region,Sector,Subsector,Energético,BNE Tcal,Emisiones BNE CO2eq (Millt)
0,Región de Tarapacá,1,Industrial y Minero,Industrias Varias,Electricidad,17.354422,NaN
1,Región de Antofagasta,2,Industrial y Minero,Industrias Varias,Electricidad,305.763385,NaN
2,Región de Atacama,3,Industrial y Minero,Industrias Varias,Electricidad,48.320890,NaN
3,Región de Coquimbo,4,Industrial y Minero,Industrias Varias,Electricidad,115.931813,NaN
4,Región de Valparaíso,5,Industrial y Minero,Industrias Varias,Electricidad,282.548859,NaN
...,...,...,...,...,...,...,...
1165,Región de Los Lagos,10,Energético: Consumo Propio,Refinería y Extracción de Petróleo y Gas Natural,Petróleo Diésel,0.199469,NaN
1166,Región de Magallanes y de la Antártica Chilena,12,Energético: Consumo Propio,Refinería y Extracción de Petróleo y Gas Natural,Petróleo Diésel,15.092826,NaN
1167,Región Metropolitana de Santiago,13,Energético: Consumo Propio,Refinería y Extracción de Petróleo y Gas Natural,Petróleo Diésel,9.326990,NaN
1168,Región de Valparaíso,5,Uso No Energético,Uso No Energético,D.i. de Petróleo,820.375073,NaN


In [53]:
def calcular_emisiones(archivo):
    #Carga de datos
    Datos_SEC = pd.read_excel(archivo, sheet_name='Datos SEC')
    UFisicas_to_Tcal = pd.read_excel(archivo,sheet_name='U. fisica a Tcal')
    BNE = pd.read_excel(archivo,sheet_name='BNE').drop(columns=['Item','Seccion'])
    BNE.columns = ['Subsector','Energético','Tcal BNE']
    FE_CO2_df = pd.read_excel(archivo,sheet_name='FE CO2').drop(columns='Poder calorifico inferior TJ/kg')
    FE_CO2_df.columns = ['Energético','Factor de emision CO2 kg/Tj']
    CE_S = pd.read_excel(archivo,sheet_name='CE S.')
    PCI_df = pd.read_excel(archivo,sheet_name='PCI')

    Datos_SEC['Sector'] = Datos_SEC['Sector'].str.lower()
    Datos_SEC['Subsector'] = Datos_SEC['Subsector'].str.lower()
    Datos_SEC['Energético'] = Datos_SEC['Energético'].str.lower()
    UFisicas_to_Tcal['Energético'] = UFisicas_to_Tcal['Energético'].str.lower()
    BNE['Subsector'] = BNE['Subsector'].str.lower()
    BNE['Energético'] = BNE['Energético'].str.lower()
    FE_CO2_df['Energético'] = FE_CO2_df['Energético'].str.lower()
    CE_S['Energético'] = CE_S['Energético'].str.lower()
    CE_S['Sector'] = CE_S['Sector'].str.lower()
    PCI_df['Energético'] = PCI_df['Energético'].str.lower()


    #Juntamos todo en una sola tabla
    data_merge = Datos_SEC.merge(UFisicas_to_Tcal,how='left',on='Energético')
    data_merge = data_merge.merge(BNE,how='left',on=['Subsector','Energético'])
    data_merge = data_merge.merge(FE_CO2_df, how='left',on='Energético')
    data_merge = data_merge.merge(CE_S,on=['Sector','Energético'],how='left')
    data_merge = data_merge.merge(PCI_df, how='left',on='Energético')
    
    # Capitalizar la primera letra de cada palabra después del merge
    data_merge['Sector'] = data_merge['Subsector'].str.title()
    data_merge['Subsector'] = data_merge['Subsector'].str.title()
    data_merge['Energético'] = data_merge['Energético'].str.title()

    #Seleccionamos variables
    SEC_UF = data_merge['SEC Unid. Físicas']
    Factor_Tcal = data_merge['Factor a Tcal'] 
    BNE_Tcal = data_merge['Tcal BNE']
    FE_CO2 = data_merge['Factor de emision CO2 kg/Tj']
    FE_CH4 = data_merge['FE CH4 (kg/TJ)']
    FE_N2O = data_merge['FE N2O (kg/TJ)']
    PCI = data_merge['ajuste a PCI']


    #Cálculos
    #Energía SEC en TCal
    SEC_Tcal = SEC_UF * Factor_Tcal
    data_merge['SEC Tcal'] = SEC_Tcal
    #Energía BNE agregado en Tcal 
    BNE_TCal_agregado = BNE_Tcal
    #Diferencia Tcal vs SEC
    data_merge['Dif %'] = np.where(BNE_Tcal > 0, (BNE_Tcal - SEC_Tcal)/BNE_Tcal, np.nan)
    #Energía BNE en TJ
    data_merge['BNE TJ'] = BNE_Tcal * 4.1868

    #Energía SEC en TJ 
    SEC_TJ = SEC_Tcal * 4.1868

    #Cálculo de emisiones SEC en (kg)
    Emisiones_SEC_CO2_kg = SEC_TJ * FE_CO2
    Emisiones_SEC_CH4_kg = SEC_TJ * FE_CH4
    Emisiones_SEC_N2O_kg = SEC_TJ * FE_N2O

    #Cálculo de emisiones SEC en (Millt)
    Emisiones_SEC_CO2_Millt = Emisiones_SEC_CO2_kg / 1000000000
    Emisiones_SEC_CH4_Millt = Emisiones_SEC_CH4_kg / 1000000000 
    Emisiones_SEC_N2O_Millt = Emisiones_SEC_N2O_kg / 1000000000 

    #Calculo de emisiones SEC en (Millt) con ajuste PCI
    Emisiones_SEC_CO2_Millt_PCI = PCI * Emisiones_SEC_CO2_Millt
    Emisiones_SEC_CH4_Millt_PCI = PCI * Emisiones_SEC_CH4_Millt
    Emisiones_SEC_N2O_Millt_PCI = PCI * Emisiones_SEC_N2O_Millt
    
    #Cálculo de emisiones CO2eq de SEC en (Millt) 
    data_merge['Emisiones SEC CO2eq (Millt)'] = Emisiones_SEC_CO2_Millt_PCI*1 + Emisiones_SEC_CH4_Millt_PCI*25 + Emisiones_SEC_N2O_Millt_PCI*298

    #Consumo de energía del BNE en TJ
    Consumo_BNE_TJ = BNE_Tcal * 4.184

    #Cálculo de emisiones BNE en (kg)
    Emisiones_BNE_CO2_kg = Consumo_BNE_TJ * FE_CO2
    Emisiones_BNE_CH4_kg = Consumo_BNE_TJ * FE_CH4
    Emisiones_BNE_N2O_kg = Consumo_BNE_TJ * FE_N2O

    #Cálculo de emisiones BNE en (Millt)
    Emisiones_BNE_CO2_Millt = Emisiones_BNE_CO2_kg / 1000000000
    Emisiones_BNE_CH4_Millt = Emisiones_BNE_CH4_kg / 1000000000
    Emisiones_BNE_N2O_Millt = Emisiones_BNE_N2O_kg / 1000000000

    #Cálculo de emisiones BNE en (Millt) con ajuste PCI
    Emisiones_BNE_CO2_Millt_PCI = PCI * Emisiones_BNE_CO2_Millt
    Emisiones_BNE_CH4_Millt_PCI = PCI * Emisiones_BNE_CH4_Millt
    Emisiones_BNE_N2O_Millt_PCI = PCI * Emisiones_BNE_N2O_Millt

    #Cálculo de emisiones CO2eq de BNE en (Millt)
    data_merge['Emisiones BNE CO2eq (Millt)'] = Emisiones_BNE_CO2_Millt_PCI*1 + Emisiones_BNE_CH4_Millt_PCI*25 + Emisiones_BNE_N2O_Millt_PCI*298

    #Consumo de energía agregado BNE en TJ
    Consumo_BNE_TJ_agregado = BNE_TCal_agregado * 4.1868

    #Cálculo de emisiones BNE agregado en (kg)
    Emisiones_BNE_CO2_kg_agregado = Consumo_BNE_TJ_agregado * FE_CO2
    Emisiones_BNE_CH4_kg_agregado = Consumo_BNE_TJ_agregado * FE_CH4
    Emisiones_BNE_N2O_kg_agregado = Consumo_BNE_TJ_agregado * FE_N2O

    #Cálculo de emisiones BNE agregado en (Millt)
    Emisiones_BNE_CO2_Millt_agregado = Emisiones_BNE_CO2_kg_agregado / 1000000000
    Emisiones_BNE_CH4_Millt_agregado = Emisiones_BNE_CH4_kg_agregado / 1000000000
    Emisiones_BNE_N2O_Millt_agregado = Emisiones_BNE_N2O_kg_agregado / 1000000000

    #Cálculo de emisiones BNE agregado en (Millt) con ajuste PCI
    Emisiones_BNE_CO2_Millt_PCI_agregado = PCI * Emisiones_BNE_CO2_Millt_agregado
    Emisiones_BNE_CH4_Millt_PCI_agregado = PCI * Emisiones_BNE_CH4_Millt_agregado
    Emisiones_BNE_N2O_Millt_PCI_agregado = PCI * Emisiones_BNE_N2O_Millt_agregado

    #Cálculo de emisiones CO2eq de BNE agregado en (Millt)
    data_merge['Emisiones BNE CO2eq (Millt) agregado'] = Emisiones_BNE_CO2_Millt_PCI_agregado * 1 + Emisiones_BNE_CH4_Millt_PCI_agregado * 25 + Emisiones_BNE_N2O_Millt_PCI_agregado * 298
    #Creamos una tabla con las columnas de interés
    data_final = data_merge[['Sector','Subsector','Energético','Unidad','SEC Tcal','Tcal BNE','Dif %','Emisiones SEC CO2eq (Millt)','Emisiones BNE CO2eq (Millt)']]

    data_merge['SEC TJ'] = SEC_TJ
    data_merge['SEC_CO2conPCI'] = Emisiones_SEC_CO2_Millt_PCI
    data_merge['SEC_CH4conPCI'] = Emisiones_SEC_CH4_Millt_PCI
    data_merge['SEC_N2OconPCI'] = Emisiones_SEC_N2O_Millt_PCI
    data_merge['SEC_CH4GK'] = Emisiones_SEC_CH4_kg
    data_merge['SEC_N2OGK'] = Emisiones_SEC_N2O_kg
    data_final.to_csv('BNE_vs_SEC_2021.csv',index=False,encoding='latin1')
    data_final = data_final.drop_duplicates()
    data_final = data_final.rename(columns={'Tcal BNE': 'BNE Tcal'})

    return data_merge.drop(columns=['Tcal BNE','Unidad','Dif %'])

In [55]:
df = calcular_emisiones(archivo)
df[pd.notna(df['SEC Tcal'])]

,Sector,Subsector,Energético,SEC Unid. Físicas,Factor a Tcal,Factor de emision CO2 kg/Tj,FE CH4 (kg/TJ),FE N2O (kg/TJ),ajuste a PCI,SEC Tcal,BNE TJ,Emisiones SEC CO2eq (Millt),Emisiones BNE CO2eq (Millt),Emisiones BNE CO2eq (Millt) agregado,SEC TJ,SEC_CO2conPCI,SEC_CH4conPCI,SEC_N2OconPCI,SEC_CH4GK,SEC_N2OGK
562,Terrestre,Terrestre,Diésel,5332.217820,9.156,74100.0,3.9,3.9,0.95,48821.786360,209623.344370,14.633852,14.997258,15.007295,204407.055132,14.389235,0.000757,7.573281e-04,797187.515014,7.971875e+05
563,Terrestre,Terrestre,Petróleo Combustible,0.460430,10.500,77400.0,0.8,0.3,0.95,4.834515,190.231353,0.00149,0.013998,0.014007,20.241147,0.001488,0.0,5.768727e-09,16.192918,6.072344e+00
564,Terrestre,Terrestre,Gasolina De Motor,4792.353840,8.176,69300.0,25,8.0,0.95,39182.284996,161935.422153,11.269066,11.11648,11.123919,164048.390821,10.800126,0.003896,1.246768e-03,4101209.770515,1.312387e+06
565,Terrestre,Terrestre,Kerosene,1.254900,8.991,71900.0,0.5,2.0,0.95,11.282806,317.721021,0.003254,0.021871,0.021886,47.238852,0.003227,0.0,8.975382e-08,23.619426,9.447770e+01
567,Terrestre,Terrestre,Gasolina Aviación,0.000000,7.980,70000.0,3,0.6,0.95,0.000000,0.701624,0.0,0.000047,0.000047,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000e+00
568,Terrestre,Terrestre,Kerosene De Aviación,0.136000,8.991,71500.0,0.5,2.0,0.95,1.222776,23.751668,0.000351,0.001626,0.001627,5.119519,0.000348,0.0,9.727085e-09,2.559759,1.023904e+01
617,Marítimo,Marítimo,Petróleo Combustible,182.084817,10.500,77400.0,0.8,0.3,0.95,1911.890583,8342.625350,0.589418,0.613889,0.6143,8004.703494,0.588586,0.000006,2.281340e-06,6403.762795,2.401411e+03
619,Marítimo,Marítimo,Kerosene,0.000000,8.991,71900.0,0.5,2.0,0.95,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000e+00
648,Aéreo,Aéreo,Gasolina Aviación,0.000000,7.980,70000.0,3,0.6,0.95,0.000000,127.063119,0.0,0.008475,0.00848,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000e+00
649,Aéreo,Aéreo,Kerosene De Aviación,1005.497991,8.991,71500.0,0.5,2.0,0.95,9040.432433,35951.834468,2.592874,2.461164,2.462811,37850.482511,2.570994,0.000018,7.191592e-05,18925.241255,7.570097e+04
